In [0]:
%sql
CREATE CATALOG resume;

In [0]:
%sql
create database resume.data;

In [0]:
%sql
create volume resume.data.volume;

In [0]:
%pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 44.3 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
import fitz  # PyMuPDF

pdf_path = "/Volumes/resume/data/volume/Resume_Akash_Verma.pdf"

doc = fitz.open(pdf_path)

text = ""
for page in doc:
    text += page.get_text()

print(text)


Akash Verma
Email: akash.verma22@outlook.com
Phone: +91 91234 56789
Location: Hyderabad, India
Education:
B.Tech IT, SRM University (2021–2025)
Skills:
Python, SQL, PySpark, Databricks, ETL
Projects:
ETL Pipeline using Medallion Architecture



In [0]:
import fitz
import os
from pyspark.sql import Row

base_path = "/Volumes/resume/data/volume"

rows = []

for file in os.listdir(base_path):
    if file.endswith(".pdf"):
        full_path = f"{base_path}/{file}"
        doc = fitz.open(full_path)

        text = ""
        for page in doc:
            text += page.get_text()

        rows.append(
            Row(
                resume_file=file,
                resume_text=text
            )
        )

df_bronze = spark.createDataFrame(rows)
df_bronze.show(truncate=False)


+-----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|resume_file            |resume_text                                                                                                                                                                                                                                                                                                                                                                                           |
+-----------------------+---------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
df_bronze.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("resume_bronze")
